<a href="https://colab.research.google.com/github/dvillasxUdg/ti_p4/blob/main/TI_1_Practica4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4

**Nombre:** Damian Alfonso Villaseñor Cisneros  
**e-mail:** damian.villasenor3846@alumnos.udg.mx

## MODULES

In [8]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

import plotly.express as px

## CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [9]:
def bm_2d(n_steps=1000, speed=6, s_pos=[0,0]):
  """
  Arguments:
    n_steps (int): number of steps to walk
    speed (float): speed of the walker
    s_pos (list): init position for [x, y].
  Returns:
    BM_2d_df(DataFrame): DataFrame with a BM
  """

  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [10]:
def crw_2d(n_steps=1000, speed=6, s_pos=[0, 0], crw_exponent=0.7):
    """
    Arguments:
        n_steps (int): number of steps to walk
        speed (float): speed of the walker
        s_pos (list): init position for [x, y].
        crw_exponent (float): wrapcauchy distribution exponent

    Returns:
        crw_2d_df(DataFrame): DataFrame with a CRW
    """
    velocity = Vec2d(speed, 0)

    crw_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

    crw_2d_df = pd.concat([crw_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        turn_angle = wrapcauchy.rvs(crw_exponent)
        velocity = velocity.rotated(turn_angle)

        s_pos = [s_pos[0] + velocity.x, s_pos[1] + velocity.y]

        temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
        crw_2d_df = pd.concat([crw_2d_df, temp_df], ignore_index=True)

    return crw_2d_df

In [11]:
def lw_2d(n_steps=1000, speed=6, s_pos=[0, 0], crw_exponent=0.9, alpha=1.9, beta=0.0, loc=3.0):
    velocity = Vec2d(speed, 0)
    levy_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    levy_2d_df = pd.concat([levy_2d_df, temp_df], ignore_index=True)

    total_steps = 0

    while total_steps < n_steps-1:
        turn_angle = wrapcauchy.rvs(crw_exponent)
        step = int(levy_stable.rvs(alpha=alpha, beta=beta, loc=loc))

        if total_steps + step > n_steps:
            step = n_steps - total_steps

        velocity = velocity.rotated(turn_angle)

        for _ in range(step):
            s_pos = [s_pos[0] + velocity.x, s_pos[1] + velocity.y]
            temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
            levy_2d_df = pd.concat([levy_2d_df, temp_df], ignore_index=True)

        total_steps += step

    return levy_2d_df

In [12]:
def path_length(df):
  dis_df = np.array([distance.euclidean(df.iloc[i-1], df.iloc[i]) for i in range(1, df.shape[0])])
  pl = np.cumsum(dis_df)
  return pl

In [13]:
def msd(df):
  MSD = np.empty(shape=(0))
  for tau in range(1, df.shape[0]):
    deltas = df.iloc[tau:].values - df.iloc[:-tau].values
    sq_distances = np.sum(deltas**2, axis=1)
    msd_value = np.mean(sq_distances)
    MSD = np.append(MSD, msd_value)
  return MSD

# UI Functions

In [14]:
def generate_trajectory(n_steps, initial_position, movement_type, crw_exponent=0.9, alpha=1.9):
    if movement_type == 'BM':
        title = 'Brownian Motion (BM)'
        return title, bm_2d(n_steps=n_steps, s_pos=initial_position, speed=step_size_slider.value)
    elif movement_type == 'CRW':
        title = 'Continuous Random Walk (CRW)'
        return title, crw_2d(n_steps=n_steps, s_pos=initial_position, speed=step_size_slider.value, crw_exponent=crw_exponent)
    elif movement_type == 'LF':
        title = 'Levy Flight (LF)'
        return title, lw_2d(n_steps=n_steps, s_pos=initial_position, speed=step_size_slider.value, crw_exponent=crw_exponent, alpha=alpha)

In [15]:
def update_trajectory(event):
    n_steps = n_steps_slider.value
    initial_position = [initial_position_x.value, initial_position_y.value, 0]
    movement_type = movement_type_selector.value

    if movement_type in ('CRW', 'LF'):
        crw_exp = crw_exponent_input.value
    else:
        crw_exp = None

    if movement_type == 'LF':
        alpha_val = alpha_input.value
    else:
        alpha_val = None

    title, trajectory = generate_trajectory(n_steps, initial_position, movement_type, crw_exponent=crw_exp, alpha=alpha_val)

    # Plot 3D
    fig_3d = px.line_3d(trajectory, x='x_pos', y='y_pos', z=trajectory.index, title=title)
    trajectory_panel.object = fig_3d

    # Plot 2D
    metric_type = metric_selector.value
    if metric_type == 'MSD':
        metric_data = msd(trajectory)
        metric_title = 'Mean Squared Displacement (MSD)'
    elif metric_type == 'PL':
        metric_data = path_length(trajectory)
        metric_title = 'Path Length (PL)'

    fig_2d = go.Figure(data=go.Scatter(x=np.arange(len(metric_data)), y=metric_data, mode='lines'))
    fig_2d.update_layout(title=metric_title, xaxis_title='Time', yaxis_title=metric_title)
    metric_panel.object = fig_2d

# Set UI

In [16]:
movement_type_selector = pn.widgets.RadioButtonGroup(name='Trajectory Type', options=['BM', 'CRW', 'LF'])
n_steps_slider = pn.widgets.IntSlider(name='Number of Steps', start=1000, end=10000, value=1000)
initial_position_x = pn.widgets.IntInput(name='xIniPos', value=0)
initial_position_y = pn.widgets.IntInput(name='yIniPos', value=0)
step_size_slider = pn.widgets.IntSlider(name='Speed', start=1, end=10, value=1)
metric_selector = pn.widgets.Select(name='Metric Type', options=['MSD', 'PL'])
crw_exponent_input = pn.widgets.FloatInput(name='CRW Exponent', start=0.1, end=1.0, step=0.1, value=0.5)
alpha_input = pn.widgets.FloatInput(name='Alpha', start=0.1, end=2.0, step=0.1, value=1.0)

# Event Listeners

In [17]:
n_steps_slider.param.watch(update_trajectory, 'value')
initial_position_x.param.watch(update_trajectory, 'value')
initial_position_y.param.watch(update_trajectory, 'value')
movement_type_selector.param.watch(update_trajectory, 'value')
crw_exponent_input.param.watch(update_trajectory, 'value')
alpha_input.param.watch(update_trajectory, 'value')
metric_selector.param.watch(update_trajectory, 'value')
step_size_slider.param.watch(update_trajectory, 'value')

Watcher(inst=IntSlider(end=10, name='Speed', start=1, value=1), cls=<class 'panel.widgets.slider.IntSlider'>, fn=<function update_trajectory at 0x77fec5dcd990>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

# Init Dashboard

In [18]:
initial_trajectory_title, initial_trajectory = generate_trajectory(n_steps_slider.value, [initial_position_x.value, initial_position_y.value, 0], movement_type_selector.value)
fig_3d = px.line_3d(initial_trajectory, x='x_pos', y='y_pos', z=initial_trajectory.index, title=initial_trajectory_title)
trajectory_panel = pn.pane.Plotly(fig_3d)

initial_metric_data = msd(initial_trajectory)
initial_metric_title = 'Mean Squared Displacement (MSD)'
fig_2d = go.Figure(data=go.Scatter(x=np.arange(len(initial_metric_data)), y=initial_metric_data, mode='lines'))
fig_2d.update_layout(title=initial_metric_title, xaxis_title='Time', yaxis_title=initial_metric_title)
metric_panel = pn.pane.Plotly(fig_2d)


# Show Panel

In [19]:
layout = pn.Row(
    pn.Column(
        movement_type_selector,
        n_steps_slider,
        initial_position_x,
        initial_position_y,
        step_size_slider,
        crw_exponent_input,
        alpha_input,
        metric_selector,
    ),
    pn.Column(
        trajectory_panel,
        width=500
    ),
    pn.Column(
        metric_panel,
        width=500
    )
)

layout.servable()

Row
    [0] Column
        [0] RadioButtonGroup(name='Trajectory Type', options=['BM', 'CRW', 'LF'], value='BM')
        [1] IntSlider(end=10000, name='Number of Steps', start=1000, value=1000)
        [2] IntInput(name='xIniPos')
        [3] IntInput(name='yIniPos')
        [4] IntSlider(end=10, name='Speed', start=1, value=1)
        [5] FloatInput(end=1.0, name='CRW Exponent', start=0.1, value=0.5)
        [6] FloatInput(end=2.0, name='Alpha', start=0.1, value=1.0)
        [7] Select(name='Metric Type', options=['MSD', 'PL'], value='MSD')
    [1] Column(width=500)
        [0] Plotly(Figure)
    [2] Column(width=500)
        [0] Plotly(Figure)